![Flight departing large city](flight-nyc.png)


A foremost aviation industry player with a significant presence in New York City has launched an in-depth data analysis project focused on identifying trends in flight durations in air travel. This initiative aims to delve into a wealth of data related to flight schedules and operational patterns, with the objective of optimizing flight times and enhancing the overall travel experience for passengers. As the head data analyst, you have access to rich datasets, sourced from the 'nycflights2022' collection produced by the ModernDive team. These datasets include records of flights departing from major New York City airports, including JFK (John F. Kennedy International Airport), LGA (LaGuardia Airport), and EWR (Newark Liberty International Airport), during the second half of 2022. They offer a comprehensive view of flight operations, covering various aspects such as departure and arrival times, flight paths, and airline specifics:

- `flights2022-h2.csv` contains information about each flight including 

| Variable         | Description                                              |
|------------------|----------------------------------------------------------|
| `carrier`        | Airline carrier code                                     | 
| `origin`         | Origin airport (IATA code)                               | 
| `dest`           | Destination airport (IATA code)                          | 
| `air_time`       | Duration of the flight in air, in minutes                |

- `airlines.csv` contains information about each airline:

| Variable  | Description                          |
|-----------|--------------------------------------|
| `carrier` | Airline carrier code                 |
| `name`    | Full name of the airline             |

- `airports.csv` provides details of airports:

| Variable | Description                           |
|----------|---------------------------------------|
| `faa`    | FAA code of the airport               |
| `name`   | Full name of the airport              |

In [19]:
# Import required packages
library(dplyr)
library(readr)

# Load the data
flights <- read_csv("flights2022-h2.csv")
airlines <- read_csv("airlines.csv")
airports <- read_csv("airports.csv")

# Start your code here!
options(dplyr.width = Inf)
print(flights)

Rows: 218802 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): carrier, tailnum, origin, dest
dbl  (14): year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, ...
dttm  (1): time_hour

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 16 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): carrier, name

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1251 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): faa, name, dst, tzone
dbl (4): lat, lon, alt, tz

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `sho

# A tibble: 218,802 × 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <dbl> <dbl> <dbl>    <dbl>          <dbl>     <dbl>    <dbl>          <dbl>
 1  2022     7     1        9           2129       160      118           2312
 2  2022     7     1       12           1940       272      315           2253
 3  2022     7     1       21           2120       181      140           2240
 4  2022     7     1       21           2159       142      225             21
 5  2022     7     1       22           2140       162      310             53
 6  2022     7     1       23           2110       193      203           2259
 7  2022     7     1       23           2100       203       NA              3
 8  2022     7     1       39           1457       582      135           1626
 9  2022     7     1       44           2155       169      134           2308
10  2022     7     1       57           1700       477      159           1829
   arr_delay carrier flight

In [20]:
start_1 <- flights %>%
group_by(carrier, dest)%>%
summarise(count_of_flights = n(),
		  avg_duration = mean(air_time, na.rm = TRUE)) %>%
ungroup()

frequent<-start_1 %>% 
left_join(airlines, by=c('carrier')) %>%
left_join(airports, by = c('dest'='faa'), suffix = c('_airline', '_airport'))%>%
select(carrier, dest, count_of_flights, avg_duration, name_airline, name_airport)%>%
rename(airline_name= name_airline, airport_name= name_airport)%>%
arrange(desc(count_of_flights))%>%
head(n=1)
frequent

`summarise()` has grouped output by 'carrier'. You can override using the
`.groups` argument.


carrier,dest,count_of_flights,avg_duration,airline_name,airport_name
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
DL,ATL,5264,109.2121,Delta Air Lines Inc.,Hartsfield Jackson Atlanta International Airport


In [21]:
start_2 <- flights %>%
mutate(air_time_hr = air_time/60) %>%
group_by(carrier, dest)%>%
summarise(avg_duration_hours=mean(air_time_hr, na.rm=TRUE))%>%
ungroup()

longest <- start_2 %>%
left_join(airlines, by=c('carrier')) %>%
left_join(airports, by=c('dest'='faa'))%>%
rename(airline_name = name.x, airport_name = name.y)%>%
select(airport_name, airline_name, avg_duration_hours)%>%
arrange(desc(avg_duration_hours))%>%
head(n=1)

`summarise()` has grouped output by 'carrier'. You can override using the
`.groups` argument.


In [22]:
start_3 <- flights %>%
mutate(air_time_hr = air_time/60) %>%
filter(origin=='JFK')%>%
group_by(origin,dest)%>%
summarise(count=n())

least <-start_3 %>%
left_join(airports, by = c('dest'='faa'))%>%
select(name, origin, dest, count)%>%
arrange(count)%>%
head(n=1)%>%
pull(name)

`summarise()` has grouped output by 'origin'. You can override using the
`.groups` argument.
